# 1 Introduction and clinical context

# 2 Defintions methods and hypotheses

# 3 Import of data and libraries

## 3.1 Import of libraries

In [ ]:
#Importing basic data analyst libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#Importing libraries for SQL querrying
import duckdb 

##3.2 Importing tables

I decided to import all the tables relevant for my analysis at this point. I will treat them as a database and later on i will look at them in detail. Now I will just import them one by one whereas querrying and cleaning comes a bit later in separate stages. I chose this approach because i find it more logically organised compared to other solutions.

###3.2.1 Importing confirmed cases table

In [ ]:
df_cases = pd.read_csv('data/raw/confirmed_global.csv')

Verifying import

In [ ]:
print(df_cases.head())

##### ---Cases dataset imported successfully---

### 3.2.2 Importing death cases

In [ ]:
df_deaths = pd.read_csv('data/raw/deaths_global.csv')

Verifying import

In [ ]:
print(df_deaths.head())

##### ----Deaths dataset importet successfully----

### 3.2.3 Importing recovered cases dataset

In [ ]:
df_recovered = pd.read_csv('data/raw/recovered_global.csv')

In [ ]:
print(df_recovered.head())

##### ----Recovered dataset importet successfully!----

# 4 Inspecting cleaning and preparing datasets for melting

In this sections I will get acquianted with dataset, melt it, deal with missings and prepare it for melting

## 4.1 Inspecting cleaning and preparing the dataset with cases

### 4.1.1 Inspecting the dataset with cases

In [ ]:
print(df_cases.head())

In [ ]:
print(df_cases.shape)

Shape of the dataset is 274 rows and 449 columns. 

From the first glance I see identifier columns such as province/state, Country/Relogion, Latitude and longitude. There are 449 columns where majority corresponds to individual dates within given period of time. To confirm that only date columns follow 'Long', and that there are no unexpected columns in between, I will iterate through the column names and count the columns to crosscheck whether i itterate through columns correctly.

In [ ]:
count = 0
for col in df_cases:
    print(col)
    count = count + 1
    
print("the column count is: ", count)

All the respective columns  after 'Long' seem to represent individual dates and column count from the loop is in agreement with the column count from df_cases.head(). Next I need to check data types of individual columns

In [ ]:
count_integer = 0
count_other_datatypes = 0
for col in df_cases:
    print(df_cases[col].dtype)
    if df_cases[col].dtype == int:
        count_integer = count_integer +1
    else:
        count_other_datatypes = count_other_datatypes + 1

print('Integer count is:', count_integer, '\n Other data types count is:', count_other_datatypes)
if count_integer == 445:
    print('All date-columns are integers!')

Loop confirmed that all date columns are formated as an integer datatype. I see that other datatypes in the dataset are objects and float. I will confirm it with command .info()

In [ ]:
df_cases.info()

Verification showed exactly those datatypes i noticed  above

In [ ]:
print(df_cases.describe)

Describe command gives me an overview over the dataset. There are a lot of missing values in province/state. I have to investigate this variable closer. Country/region column will likely be the variable of interest and will serve us as identifier, Whereas latitute and longitude are not interesting forour analysis. Even though some viruses are known to spread better in colder or warmer climates, Cross-country comparisons in cases between the countries are not feasible because we don´t have data on people tested and countries were very different in test measures and  test equipment. In this dataset we lack the data for it. I will not check dataset with .describe(). Quick overview shows that there were 0 cases in a begining whereas number of cases began to increase and fluctuate over time.

### 4.1.2 Cleaning the dataset with cases

I am checking if there are any missing values in columns

In [ ]:
df_cases.isna()

I cannot see the whole dataset, but the output indicates that Province/state has missing values. I am going to investigate it further along with other columns

In [ ]:
df_cases.isna().any()

It seems that longitude latitude and province contain misssing values whereas dates seem to contain no missing value. I need to check my assumption.

In [ ]:
df_cases.isna().any().sum()


There are three columns in the df_cases. They are Province/State, Lat, Long.

There are three columns in the df_cases. They are Province/State, Lat, Long.
Before I go further with cleaning and droping, want to check which column is the most pertinent one to become identifier column during melting. The two most valiable options are Province/state and Country/Region. I will extend the jupyter notebook output to contain up to 500 rows and columns which should be sufficient size to cover the entire df_cases dataframe.

In [ ]:
#extending output for rows
pd.set_option('display.max_rows', 300)
#extending output for columns
pd.set_option('display.max_columns', 500)

In [ ]:
print(df_cases[['Province/State', 'Country/Region']])

I confirm that column 'Province/State' is of no interest as it shows either overseas territories and provinces/states within a few countries.Contrary, Norway and Slovakia are both within Country/Region column which will become our identifier. Now I will strip the dataset from all the unneccesarry columns and rows.

In [ ]:
df_cases_filtered = """
SELECT *
FROM df_cases
WHERE "Country/Region" = 'Norway' OR "Country/Region" = 'Slovakia';
"""
df_cases = duckdb.query(df_cases_filtered).to_df()

Checking whether i filtered rows correctly

In [ ]:
print(df_cases.iloc[:, 1:])
print(type(df_cases), df_cases.shape)



I correctly filtered just relevant rows and transformed querried data into dataframe. There are two rows- Slovakia and Norway with number of columns(449) remained unchanged. Now I will proceed with dropping all the unneccessarry columns. namely: 
'Province/State', 
'Long', 
'Lat'

As mentioned earlier, variables lattitude and longitude are of  no value in our analysis, whereas province/state is variable indicating a region within larger countries.Countries of interest are slovakia and Norway and we have no addotional data on specific locations within those countries.

In [ ]:
df_cases.drop(['Province/State', 'Long','Lat',], axis = 1, inplace = True)

Checking if there are some missing values in the dataset:
df_cases_filtered.isna()

In [ ]:

print(df_cases,df_cases.shape)
print('', df_cases.isna().any().sum())

I correctly dropped all the unneccessarry rows and dataset is ready to melt. But before I will do it I will prepare datasets with deaths and recovered cases.

## 4.2 Inspecting cleaning and preparing the dataset with deaths

### 4.2.1 Inspecting the dataset with deaths

In [ ]:
print(df_deaths.shape)

The deaths dataset has the same number of rows(274) and columns (449) as the cases dataset. its a good indication because the process of cleaning checking and extracting of data will be very simmilar.

In [ ]:
print(df_deaths.head())

Again, earlier dates indicate zero dates whereas number of deaths towards the later period is higher and steadily increasing Whereas first five variables: 'Province/State' 'Country/Region 'Lat''Long' seem to be identical through all the datasets. I have to confirm this assumption

I am checking what datatypes exist in the dataset deaths

In [ ]:
df_deaths.info()

df_deaths dataset seem to have identical types  two float columns (lat, long), two object columns (Province/State Country/Region) and 445 integer(date columns). I will verify it

In [ ]:
print(df_deaths['Country/Region'].dtype)
print(df_deaths['Country/Region'].dtype)
print(df_deaths['Long'].dtype)
print(df_deaths['Lat'].dtype)

I confirm my assumption about float and object columns, I will check the date(integer columns as well)

In [ ]:
#Filtering dataset so there are only dates
df_deaths_only_date = df_deaths.iloc[:, 4:]
#Creating loop to check which type the column is
date_counter = 0
for col in df_deaths_only_date:
    print(df_deaths_only_date[col].dtype)
    date_counter +=1 

print(date_counter)
print(df_deaths_only_date.info())

I confirm that the dataset contains 445 integers! I am finished with scrutinizing the dataset with deaths and I move further to clean it.

### 4.2.2 Cleaning the dataset with deaths

I first check whether columns contain missing values

In [ ]:
print(df_deaths.isna().any())
df_deaths.isna().any().sum()



#I will check how many columns with missing values are present. if there are three we know they are those:
 Province/State      
 Lat       
 Long

In [ ]:
df_deaths.isna().any().sum()

I confirm the assumption about the missing values. Now I check identifier columns

In [ ]:
print(df_deaths['Province/State'],df_deaths['Country/Region'])

I will drop the Long, Lat and Province/State columns as they are of no interest for us:

In [ ]:
df_deaths.drop(['Province/State', 'Long', 'Lat'], axis = 1, inplace = True)

Filtering rows with Slovakia and Norway and re-writing original dataset:

In [ ]:
df_deaths.head()

In [ ]:
#row-filtering
df_deaths_filtered = """
SELECT * FROM df_deaths
WHERE "Country/Region" = 'Slovakia' OR "Country/Region" = 'Norway' """
#re-writing df_deaths
df_deaths = duckdb.query(df_deaths_filtered).to_df()

In [ ]:

print(' The dataset has', df_deaths.isna().any().sum(), 'missings', df_deaths.shape, df_deaths.info())

In [ ]:
print(df_deaths)

Dataset is correctly cleaned and filtered with dates as integers, identifier column som object and zero missing values.

## 4.3 Inspecting cleaning and preparing the dataset with recovered cases

### 4.3.1 Inspecting the dataset with recovered cases

In [ ]:
df_recovered.head()

The dataset seems to follow the exactly same pattern as the two  datasets above.

In [ ]:
df_recovered.info()

In [ ]:
df_recovered.dtypes

Contrary to other columns variable lat is coded as an object. It doesn´t have an effect on my decision to drop it, it is just peculiar observation. Now I check whether there is  Norway and Slovakia in country/region variable.

In [ ]:
#Checking country column
print(df_recovered['Country/Region'].isin(['Norway']).any())
print(df_recovered['Country/Region'].isin(['Slovakia']).any())


#Checking state column

print(df_recovered["Province/State"].isin(["Slovakia"]).any())
print(df_recovered['Province/State'].isin(['Norway']).any())

We did find both countries in Country/Region, Whereas we didnt find any of the countries of interest in Province/State. This means again that identifier will be Country/Region. I will confirm that dates are integers in subsequent code:

In [ ]:
recovered_dates = df_recovered.iloc[:,4:]

In [ ]:
recovered_dates.info()


All dates are integers

### 4.3.2 Cleaning the dataset with recovered cases

I will filter the rows dataset applying boolean mask to relevant rows and columns:

In [ ]:
df_recoveredd =df_recovered[(df_recovered['Country/Region'] == 'Slovakia') | (df_recovered['Country/Region'] == 'Norway')].copy()

In [ ]:
#Reassigning the new table to the old one
df_recovered = df_recoveredd
df_recovered

I need to change row indexes. I reset them.

In [ ]:
df_recovered.reset_index(drop = True, inplace = True)

Now I  drop all the irelevant columns

In [ ]:
#Droping unneccessarry columns
df_recovered.drop(['Long', 'Lat', 'Province/State'], axis = 1, inplace = True)
df_recovered

Cleaning successfull!

## 4.4 Checking the tables all together

In [ ]:
df_cases

In [ ]:
df_deaths

In [ ]:
df_recovered

All the tables seem to be correctly filtered, I confirm that by printing out datatypes in columns of respective dataframes.

In [ ]:
print(df_recovered.info())
print(df_deaths.info())
print(df_cases.info())



All the dataframes have 445 integers(dates) and 1 object(identifier). I will finally confirm a cleanliness of dataset viewing their shape

In [ ]:
print(df_recovered.shape)
print(df_deaths.shape)
print(df_cases.shape)

All of them have two rows(Slovakia and Norway) and 446 columns out of which 445 are date columns and 1 is identifier. I confirm dataset is ready for further analysis.

# 5 Melting the datasets

## 5.1 Melting the cases dataset

Melting means that I change format of the table. Instead of having 446 columns, I want to have just three. these are:
country/region, date and number of cases.The table will be long because each date column will be listed as row twice- one for slovakia and same date for norway. 

In [ ]:
df_cases = df_cases.melt(id_vars = 'Country/Region', var_name = 'date', value_name = "cases_number").copy()

Checking the table

In [ ]:
df_cases

The table is melted correctly, there are 890 rows which is 455 per country this number of rows corresponds to number of date columns in the original dataset

In [ ]:
## 5.2 Melting the deaths dataset

In [ ]:
df_deaths = df_deaths.melt(id_vars = "Country/Region", var_name = 'date', value_name = 'deaths_number').copy()

Checking the table

In [ ]:
df_deaths

I confirm that columns are named correctly and shape of the table 890 x 3 is desirable.

In [ ]:
## 5.3 Melting the dataset with recovered cases

In [ ]:
df_recovered = df_recovered.melt(id_vars = "Country/Region", var_name = "date", value_name = 'recovered_number').copy()

Checking the table

In [ ]:
df_recovered

I confirm that table shape matches perfectly the table shapes of recovered and cases tables. All three tables begin and finish with the same date. Lastly i check if there are any duplicates in the date columns. Each dataset should not have more than 2 duplicates of date(corresponding to 2 respective countries) in itself.

Accessing the number of duplicates for each date in each dataset(correct should be 2 for each date)

In [ ]:
cases_duplicates_size = df_cases.groupby("date").size()
deaths_duplicates_size = df_deaths.groupby("date").size()
recovered_duplicates_size = df_recovered.groupby("date").size()

Checking if each date contains no more or no less than two duplicates:

In [ ]:

print((cases_duplicates_size == 2).all())
print((deaths_duplicates_size == 2).all())
print((recovered_duplicates_size == 2).all())

Melting successfull!

6. Merging the datasets

In [ ]:
Merging datasets using SQL querying

In [ ]:
df_timeseries = """SELECT c."Country/Region" AS country,
                          c."cases_number", 
                          d."deaths_number", 
                          r."recovered_number"
FROM df_cases
LEFT JOIN (SELECT Country/Region,
                  date	
                  deaths_number FROM df_deaths) AS d
LEFT JOIN df_recovered AS r"""

In [ ]:
Transforming query to dataframe

In [ ]:
df_timeseries = duckdb.query(timeseries).to_df()

Checking integrity of the merged dataframe

In [ ]:
df_timeseries